In [ ]:
"""
This module imports necessary libraries and defines the intrinsic camera parameters, 
loads a set of images, and implements a function to match keypoints between two images using the SIFT algorithm.

Imports:
- numpy: For numerical operations and array handling.
- cv2: OpenCV library for computer vision tasks.

Camera Intrinsic Parameters:
- K: A 3x3 matrix representing the intrinsic parameters of the camera, including focal lengths and optical center.

Image Loading:
- image_paths: A list of file paths for the images to be processed, generated using a formatted string for file naming.

Function: get_matches(path1, path2)
-------------------------------------
This function takes in the paths of two images, performs the following operations:
1. Reads the images from the specified paths.
2. Checks if the images are loaded correctly; if not, it logs an error message.
3. Uses the SIFT algorithm to detect keypoints and compute their descriptors.
4. Sets up the FLANN-based matcher to find the best matches between the descriptors of the two images.
5. Applies Lowe's ratio test to filter out weak matches.
6. Collects matched keypoints' coordinates and their corresponding colors from both images.

Parameters:
- path1 (str): The file path for the first image.
- path2 (str): The file path for the second image.

Returns:
- matched_points_a (np.ndarray): Coordinates of matched keypoints in the first image.
- matched_points_b (np.ndarray): Coordinates of matched keypoints in the second image.
- total_colors_a (np.ndarray): Color values of matched keypoints in the first image.
- total_colors_b (np.ndarray): Color values of matched keypoints in the second image.
"""

In [18]:
import numpy as np
import cv2 as cv

# Camera intrinsic parameters
K = np.array([
    [1270.95, 0.00032, 518.57],
    [0, 1272.75, 393.43],
    [0, 0, 1]
])

# Loading images 
image_paths = [f'./Assets/{i:04d}.jpg' for i in range(8)]
sift = cv.SIFT_create()


def get_matches(path1,path2):
    img1 = cv.imread(path1)
    img2 = cv.imread(path2)
        # Initialize lists to store colors
    total_colors_a = []  # Initialize as lists
    total_colors_b = []

    # Ensure the images are read correctly
    if img1 is None or img2 is None:
        print(f"Error reading images")
        return 
    
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)
    
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=50)  # or pass empty dictionary
    flann = cv.FlannBasedMatcher(index_params, search_params)
    
    matches = flann.knnMatch(des1, des2, k=2)
    
    # Need to draw only good matches, so create a mask
    matchesMask = [[0, 0] for _ in range(len(matches))]
    
    # Ratio test as per Lowe's paper
    for j, (m, n) in enumerate(matches):
        if m.distance < 0.7 * n.distance:
            matchesMask[j] = [1, 0]

    # Lists to store coordinates of matched keypoints
    matched_points_a = []
    matched_points_b = []

    # Iterate through the matches and their mask
    for j, (m, n) in enumerate(matches):
        if matchesMask[j][0] == 1:  # Only consider good matches
            # Get the coordinates from keypoints
            point_a = kp1[m.queryIdx].pt  # Coordinates in Image A
            point_b = kp2[m.trainIdx].pt  # Coordinates in Image B
            
            # Append the coordinates to the lists
            matched_points_a.append(point_a)  # (x, y) coordinates in Image A
            matched_points_b.append(point_b)  # (x, y) coordinates in Image B

            # Extract colors from the images based on matched feature points
            color_a = img1[int(point_a[1]), int(point_a[0])]  # Color from Image A
            color_b = img2[int(point_b[1]), int(point_b[0])]  # Color from Image B
            
            # Store colors in total colors lists
            total_colors_a.append(color_a.tolist())  # Convert to list before appending
            total_colors_b.append(color_b.tolist())  # Convert to list before appending
    return np.array(matched_points_a) , np.array(matched_points_b) , np.array(total_colors_a) , np.array(total_colors_b)


In [ ]:
"""
This cell initializes transformation matrices and computes the initial camera poses based on the camera's intrinsic parameters. 
It also retrieves matched feature points and their corresponding colors from two consecutive images.

Key Variables:
- transform_matrix_0: A 3x4 transformation matrix representing the initial pose of the first camera. 
                      It is initialized to the identity matrix for the rotational part and zero for the translational part.
- transform_matrix_1: A 3x4 empty matrix that will be populated later to represent the pose of the second camera.
- pose_0: A 3x4 matrix representing the camera pose of the first image, computed by multiplying the intrinsic matrix K with transform_matrix_0.
- pose_1: A 3x4 empty matrix that will later hold the computed pose of the second image.
- feature_0: 2D coordinates of matched keypoints in the first image retrieved by the get_matches function.
- feature_1: 2D coordinates of matched keypoints in the second image retrieved by the get_matches function.
- color_a: Color values of matched keypoints from the first image.
- color_b: Color values of matched keypoints from the second image.

Operations:
1. Initializes the transformation matrices to prepare for camera pose estimation.
2. Calculates the pose of the first camera using the intrinsic parameters.
3. Calls the get_matches function to obtain matched keypoints and their colors from the first two images in the dataset.
"""

In [2]:
transform_matrix_0 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0]])
transform_matrix_1 = np.empty((3, 4))
    
pose_0 = np.matmul(K, transform_matrix_0)
pose_1 = np.empty((3, 4)) 
feature_0, feature_1,color_a,color_b = get_matches(image_paths[0], image_paths[1])


In [ ]:
"""
This cell computes the essential matrix from the matched feature points between the first and second images,
recovers the camera pose from this essential matrix, and updates the transformation matrices accordingly.

Key Variables:
- essential_matrix: The computed essential matrix that encodes the relative pose between the two cameras based on the matched feature points.
- em_mask: A mask indicating which matches are inliers, obtained from the RANSAC method used to estimate the essential matrix.
- rot_matrix: The rotation component of the camera pose, extracted from the essential matrix.
- tran_matrix: The translation component of the camera pose, extracted from the essential matrix.
- transform_matrix_1: A 3x4 transformation matrix representing the pose of the second camera, updated based on the recovered rotation and translation.
- pose_1: A 3x4 matrix representing the camera pose of the second image, calculated by multiplying the intrinsic matrix K with transform_matrix_1.

Operations:
1. Computes the essential matrix using the RANSAC method and refines the matched feature points to keep only inliers.
2. Recovers the rotation and translation matrices from the essential matrix.
3. Updates the transformation matrix for the second camera pose using the recovered rotation and translation.
4. Calculates the final pose of the second camera by applying the intrinsic camera parameters to the updated transformation matrix.
"""


In [3]:

essential_matrix, em_mask = cv.findEssentialMat(feature_0, feature_1, K, method=cv.RANSAC, prob=0.999, threshold=0.4, mask=None)
feature_0 = feature_0[em_mask.ravel() == 1]
feature_1 = feature_1[em_mask.ravel() == 1]
color_a=color_a[em_mask.ravel() == 1]
color_b=color_b[em_mask.ravel() == 1]

_, rot_matrix, tran_matrix, em_mask = cv.recoverPose(essential_matrix, feature_0, feature_1,K)
feature_0 = feature_0[em_mask.ravel() > 0]
feature_1 = feature_1[em_mask.ravel() > 0]
color_a=color_a[em_mask.ravel() > 0]
color_b=color_b[em_mask.ravel() > 0]
transform_matrix_1[:3, :3] = np.matmul(rot_matrix, transform_matrix_0[:3, :3])
transform_matrix_1[:3, 3] = transform_matrix_0[:3, 3] + np.matmul(transform_matrix_0[:3, :3], tran_matrix.ravel())

pose_1 = np.matmul(K, transform_matrix_1)


In [ ]:
"""
Triangulates 3D points from two sets of 2D image points and their corresponding projection matrices.

Parameters:
- point_2d_1 (numpy.ndarray): A 2D array of shape (N, 2) containing the (x, y) coordinates of points from the first image.
- point_2d_2 (numpy.ndarray): A 2D array of shape (N, 2) containing the (x, y) coordinates of points from the second image.
- projection_matrix_1 (numpy.ndarray): A 3x4 projection matrix corresponding to the first camera, used for projecting 3D points to 2D.
- projection_matrix_2 (numpy.ndarray): A 3x4 projection matrix corresponding to the second camera, used for projecting 3D points to 2D.

Returns:
- projection_matrix_1.T (numpy.ndarray): The transposed projection matrix of the first camera.
- projection_matrix_2.T (numpy.ndarray): The transposed projection matrix of the second camera.
- point_cloud (numpy.ndarray): A 4xN array of homogeneous coordinates of the triangulated 3D points. The points are normalized by dividing by the fourth coordinate to obtain Cartesian coordinates.

Note:
The triangulated points are computed using the OpenCV function `cv.triangulatePoints`, which combines the two sets of 2D points and their respective projection matrices to produce the 3D point cloud.
"""


In [4]:
def triangulation(point_2d_1, point_2d_2, projection_matrix_1, projection_matrix_2):
        
        '''
        Triangulates 3d points from 2d vectors and projection matrices
        returns projection matrix of first camera, projection matrix of second camera, point cloud 
        '''


        pt_cloud = cv.triangulatePoints(point_2d_1, point_2d_2, projection_matrix_1.T, projection_matrix_2.T)
        return projection_matrix_1.T, projection_matrix_2.T, (pt_cloud / pt_cloud[3])    

In [ ]:
"""
Estimates the pose of an object from 3D-2D point correspondences using the RANSAC algorithm.

Parameters:
- obj_point (numpy.ndarray): A 3D array of shape (N, 3) containing the object points in the world coordinate system.
- image_point (numpy.ndarray): A 2D array of shape (N, 2) containing the corresponding 2D points in the image coordinate system.
- K (numpy.ndarray): A 3x3 camera intrinsic matrix that defines the camera parameters.
- rot_vector (numpy.ndarray): A 1D array representing the initial rotation vector of shape (3,)

Returns:
- rot_matrix (numpy.ndarray): A 3x3 rotation matrix derived from the estimated rotation vector.
- tran_vector (numpy.ndarray): A 3x1 translation vector that describes the position of the object in the camera coordinate system.
- image_point (numpy.ndarray): The filtered array of 2D image points that correspond to inliers found by RANSAC.
- obj_point (numpy.ndarray): The filtered array of 3D object points corresponding to the inliers.
- rot_vector (numpy.ndarray): The rotation vector corresponding to the inliers.

Note:
The function utilizes OpenCV's `cv.solvePnPRansac` to estimate the pose, robustly handling outliers. It also converts the rotation vector to a rotation matrix using `cv.Rodrigues`.
"""


"\nEstimates the pose of an object from 3D-2D point correspondences using the RANSAC algorithm.\n\nParameters:\n- obj_point (numpy.ndarray): A 3D array of shape (N, 3) containing the object points in the world coordinate system.\n- image_point (numpy.ndarray): A 2D array of shape (N, 2) containing the corresponding 2D points in the image coordinate system.\n- K (numpy.ndarray): A 3x3 camera intrinsic matrix that defines the camera parameters.\n- rot_vector (numpy.ndarray): A 1D array representing the initial rotation vector of shape (3,).\n\nReturns:\n- rot_matrix (numpy.ndarray): A 3x3 rotation matrix derived from the estimated rotation vector.\n- tran_vector (numpy.ndarray): A 3x1 translation vector that describes the position of the object in the camera coordinate system.\n- image_point (numpy.ndarray): The filtered array of 2D image points that correspond to inliers found by RANSAC.\n- obj_point (numpy.ndarray): The filtered array of 3D object points corresponding to the inliers.\n

In [5]:
def PnP( obj_point, image_point , K, rot_vector) :
        
        '''
        Finds an object pose from 3D-2D point correspondences using the RANSAC scheme.
        returns rotational matrix, translational matrix, image points, object points, rotational vector
        '''


        _, rot_vector_calc, tran_vector, inlier = cv.solvePnPRansac(obj_point, image_point, K, np.zeros((5, 1), dtype=np.float32), cv.SOLVEPNP_ITERATIVE)
        rot_matrix, _ = cv.Rodrigues(rot_vector_calc)

        if inlier is not None:
            image_point = image_point[inlier[:, 0]]
            obj_point = obj_point[inlier[:, 0]]
            rot_vector = rot_vector[inlier[:, 0]]
        return rot_matrix, tran_vector, image_point, obj_point, rot_vector

In [ ]:
"""
Calculates the reprojection error, which is the distance between the projected 3D points and the corresponding 2D points in the image.

Parameters:
- obj_points (numpy.ndarray): A 3D array of shape (N, 3) representing the object points in the world coordinate system.
- image_points (numpy.ndarray): A 2D array of shape (N, 2) containing the corresponding 2D points in the image coordinate system.
- transform_matrix (numpy.ndarray): A 4x4 transformation matrix that contains the rotation and translation information.
- K (numpy.ndarray): A 3x3 camera intrinsic matrix that defines the camera parameters.
- homogenity (int): A flag to indicate if the input object points are in homogeneous coordinates; `1` if true, `0` otherwise.

Returns:
- total_error (float): The average reprojection error normalized by the number of projected points.
- obj_points (numpy.ndarray): The original object points passed to the function, possibly in converted homogeneous coordinates.

Note:
The function projects the 3D object points onto the image plane using the provided transformation and intrinsic camera parameters, then calculates the reprojection error using the L2 norm.
"""


In [6]:
def reprojection_error(obj_points, image_points, transform_matrix, K, homogenity):

        '''
        Calculates the reprojection error ie the distance between the projected points and the actual points.
        returns total error, object points
        '''
        rot_matrix = transform_matrix[:3, :3]
        tran_vector = transform_matrix[:3, 3]
        rot_vector, _ = cv.Rodrigues(rot_matrix)
        if homogenity == 1:
            obj_points = cv.convertPointsFromHomogeneous(obj_points.T)
        image_points_calc, _ = cv.projectPoints(obj_points, rot_vector, tran_vector, K, None)
        image_points_calc = np.float32(image_points_calc[:, 0, :])
        total_error = cv.norm(image_points_calc, np.float32(image_points.T) if homogenity == 1 else np.float32(image_points), cv.NORM_L2)
        return total_error / len(image_points_calc), obj_points

In [ ]:
"""
Finds the common points between two pairs of image points: image 1 and image 2, and image 2 and image 3.

Parameters:
- image_points_1 (numpy.ndarray): A 2D array of shape (N, 2) containing the points from image 1.
- image_points_2 (numpy.ndarray): A 2D array of shape (M, 2) containing the points from image 2.
- image_points_3 (numpy.ndarray): A 2D array of shape (P, 2) containing the points from image 3.

Returns:
- cm_points_1 (numpy.ndarray): An array of indices of common points in image 1 that correspond to image 2.
- cm_points_2 (numpy.ndarray): An array of indices of common points in image 2 that correspond to image 1.
- mask_array_1 (numpy.ndarray): A 2D array of shape (K, 2) containing the points from image 2 that are not common to image 1.
- mask_array_2 (numpy.ndarray): A 2D array of shape (L, 2) containing the points from image 3 that are not common to image 2.

Note:
This function identifies common keypoints between the first two images and between the second and third images by comparing the coordinates. It creates masks for points in the second and third images that do not have corresponding points in the first and second images, respectively.
"""


In [9]:
def common_points( image_points_1, image_points_2, image_points_3) :
        '''
        Finds the common points between image 1 and 2 , image 2 and 3
        returns common points of image 1-2, common points of image 2-3, mask of common points 1-2 , mask for common points 2-3 
        '''
        cm_points_1 = []
        cm_points_2 = []
        for i in range(image_points_1.shape[0]):
            a = np.where(image_points_2 == image_points_1[i, :])
            if a[0].size != 0:
                cm_points_1.append(i)
                cm_points_2.append(a[0][0])

        mask_array_1 = np.ma.array(image_points_2, mask=False)
        mask_array_1.mask[cm_points_2] = True
        mask_array_1 = mask_array_1.compressed()
        mask_array_1 = mask_array_1.reshape(int(mask_array_1.shape[0] / 2), 2)

        mask_array_2 = np.ma.array(image_points_3, mask=False)
        mask_array_2.mask[cm_points_2] = True
        mask_array_2 = mask_array_2.compressed()
        mask_array_2 = mask_array_2.reshape(int(mask_array_2.shape[0] / 2), 2)
        print(" Shape New Array", mask_array_1.shape, mask_array_2.shape)
        return np.array(cm_points_1), np.array(cm_points_2), mask_array_1, mask_array_2

In [ ]:
"""
Triangulates 3D points from 2D correspondences between two camera poses.

Parameters:
- pose_0 (numpy.ndarray): The projection matrix for the first camera, shape (3, 4).
- pose_1 (numpy.ndarray): The projection matrix for the second camera, shape (3, 4).
- feature_0 (numpy.ndarray): A 2D array of points from the first image, shape (N, 2).
- feature_1 (numpy.ndarray): A 2D array of points from the second image, shape (M, 2).

Returns:
- points_3d (numpy.ndarray): A 3D array of triangulated points, shape (K, 3), where K is the number of 3D points.

Notes:
1. The triangulation function estimates the 3D coordinates of points based on their 2D projections in two images using the provided camera poses.
2. The output `points_3d` is converted from homogeneous coordinates to regular 3D coordinates. The conversion process normalizes the points so that they are represented in Cartesian coordinates (x, y, z).
3. The final shape of `points_3d` is (K, 3), where K corresponds to the number of successfully triangulated points.
"""


In [10]:
_, _, points_3d = triangulation(pose_0, pose_1, feature_0, feature_1)
points_3d = cv.convertPointsFromHomogeneous(points_3d.T)
points_3d = points_3d[:, 0, :]


In [ ]:
"""
Iteratively computes the 3D structure from a sequence of images using triangulation and pose estimation.

The code iterates through a series of images, computing and refining the 3D point cloud and associated color data from consecutive image pairs.

Parameters:
- pose_array (numpy.ndarray): Array to store the camera poses as flattened vectors.
- total_points (numpy.ndarray): Array containing the current 3D point cloud, initially populated with previous triangulated points.
- total_colors (numpy.ndarray): Array holding the colors corresponding to the 3D points.

Loop through the images:
1. For each pair of consecutive images, match features using the `get_matches` function.
2. Triangulate the 3D points from the matched features using the current and previous poses.
3. Convert the 3D points from homogeneous coordinates to Cartesian coordinates.
4. Identify common points between the matched features of the consecutive images.
5. Estimate the camera pose using the PnP algorithm with RANSAC to reduce the effect of outliers.
6. Compute the reprojection error to evaluate the accuracy of the estimated points.
7. Triangulate common points again for better accuracy.
8. Store the camera poses, 3D points, and colors of the points for visualization and further processing.

Returns:
- None: The function updates the `pose_array`, `total_points`, and `total_colors` arrays in place.

Notes:
1. The use of tqdm provides a visual progress bar for the iteration, indicating how many images have been processed.
2. The reprojection error helps monitor the accuracy of the 3D reconstruction process.
3. The code assumes that `K`, `pose_0`, and `pose_1` are defined and valid camera matrices for the corresponding images.
4. The loop iterates from the first image to the second last image in the `image_paths`, ensuring that pairs of images are processed.
"""


In [11]:
from tqdm import tqdm
pose_array = K.ravel()
total_points = points_3d
total_colors =color_b




for i in tqdm(range(len(image_paths)-2)):
        image = cv.imread(image_paths[i + 1])  # Read the current next image
        feature_0, feature_1, _, _ = get_matches(image_paths[i], image_paths[i+1])
        feature_0, feature_1, points_3d = triangulation(pose_0, pose_1, feature_0, feature_1)
        feature_1 = feature_1.T
        features_cur, features_2, _, _ = get_matches(image_paths[i+1], image_paths[i+2])
        points_3d = cv.convertPointsFromHomogeneous(points_3d.T)
        points_3d = points_3d[:, 0, :]
        cm_points_0, cm_points_1, cm_mask_0, cm_mask_1 = common_points(feature_1, features_cur, features_2)
        
        cm_points_2 = features_2[cm_points_1]
        cm_points_cur = features_cur[cm_points_1]
        rot_matrix, tran_matrix, cm_points_2, points_3d, cm_points_cur = PnP(
                    points_3d[cm_points_0], cm_points_2, K, 
                     cm_points_cur
                )
        transform_matrix_1 = np.hstack((rot_matrix, tran_matrix))
        pose_2 = np.matmul(K, transform_matrix_1)

                # Calculate reprojection error
        error, points_3d = reprojection_error(points_3d, cm_points_2, transform_matrix_1, K, homogenity=0)

                # Triangulate common points between poses
        cm_mask_0, cm_mask_1, points_3d = triangulation(pose_1, pose_2, cm_mask_0, cm_mask_1)
        error, points_3d = reprojection_error(points_3d, cm_mask_1, transform_matrix_1, K, homogenity=1)
        print("Reprojection Error: ", error)

                # Collect 3D points and color information
        pose_array = np.hstack((pose_array, pose_2.ravel()))
        total_points = np.vstack((total_points, points_3d[:, 0, :]))
        points_left = np.array(cm_mask_1, dtype=np.int32)
        color_vector = np.array([image[l[1], l[0]] for l in points_left.T])
        total_colors = np.vstack((total_colors, color_vector))
        pose_0 = np.copy(pose_1)
        pose_1 = np.copy(pose_2)



  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:03<00:18,  3.64s/it]

 Shape New Array (440, 2) (440, 2)
(490, 3)
(490, 2)
(5, 1)
Reprojection Error:  1.683977159738542


 33%|███▎      | 2/6 [00:09<00:19,  4.78s/it]

 Shape New Array (951, 2) (951, 2)
(565, 3)
(565, 2)
(5, 1)
Reprojection Error:  1.0456964589718885


...

 Shape New Array (710, 2) (710, 2)
(655, 3)
(655, 2)
(5, 1)
Reprojection Error:  7.6619861787075525


 83%|████████▎ | 5/6 [00:23<00:04,  4.72s/it]

 Shape New Array (850, 2) (850, 2)
(920, 3)
(920, 2)
(5, 1)
Reprojection Error:  0.633565757669806


100%|██████████| 6/6 [00:28<00:00,  4.78s/it]

 Shape New Array (711, 2) (711, 2)
(1043, 3)
(1043, 2)
(5, 1)
Reprojection Error:  53.73645788390992


In [ ]:

    # Generates a .ply file for visualizing the 3D point cloud.

    # Parameters:
    # - path (str): The directory path where the .ply file will be saved.
    # - point_cloud (numpy.ndarray): A 3D array of shape (N, 3) representing the 3D points.
    # - colors (numpy.ndarray): A 2D array of shape (N, 3) representing the RGB colors for each point, in the range [0, 1].

    # The function performs the following steps:
    # 1. Reshapes and scales the 3D point cloud.
    # 2. Converts the colors to 8-bit unsigned integers (0-255) suitable for .ply format.
    # 3. Combines the point coordinates and colors into a single array.
    # 4. Centers the points by subtracting the mean position.
    # 5. Applies a distance threshold to filter out points that are too far from the center.
    # 6. Constructs the .ply file header with the appropriate format specifications.
    # 7. Creates a 'res' directory within the specified path if it does not already exist.
    # 8. Writes the point cloud data to the .ply file.

    # Returns:
    # - None: The function saves the output .ply file to the specified directory.

    # Notes:
    # - The scaling factor of 200 is applied to the points; adjust this value based on the desired size in the visualization.
    # - The distance threshold is calculated as the mean distance plus an offset of 300, which helps in filtering the points based on their spatial distribution.



In [14]:
import os

def to_ply(path, point_cloud, colors):
    '''
    Generates the .ply which can be used to open the point cloud
    '''
    # Reshape and scale the point cloud
    out_points = point_cloud.reshape(-1, 3) * 200  # Scale points if needed

    # Convert the colors to the correct format
    # Ensure the colors are scaled to 0-255
    out_colors = (colors.reshape(-1, 3) * 255).astype(np.uint8)  # Convert to uint8

    verts = np.hstack([out_points, out_colors])

    # Center the points and apply a distance threshold
    mean = np.mean(verts[:, :3], axis=0)
    scaled_verts = verts[:, :3] - mean
    dist = np.sqrt(scaled_verts[:, 0] ** 2 + scaled_verts[:, 1] ** 2 + scaled_verts[:, 2] ** 2)
    indx = np.where(dist < np.mean(dist) + 300)
    verts = verts[indx]

    ply_header = '''ply
    format ascii 1.0
    element vertex %(vert_num)d
    property float x
    property float y
    property float z
    property uchar red
    property uchar green
    property uchar blue
    end_header
    '''

    # Create the 'res' directory if it doesn't exist
    output_dir = os.path.join(path, 'res')
    os.makedirs(output_dir, exist_ok=True)

    # Save the output ply file
    output_file = os.path.join(output_dir, 'point_cloud.ply')  # Naming the file point_cloud.ply
    with open(output_file, 'w') as f:
        f.write(ply_header % dict(vert_num=len(verts)))
        np.savetxt(f, verts, '%f %f %f %d %d %d')


(4679, 3)
(4679, 3)


In [17]:
to_ply('./Assets', total_points, total_colors)  # Using './Assets' as the base path for saving
